> ## ❓Questions
> - What are some newer approaches to ML?
> - What are their pros and cons?
>
> ## ☑︎ Objectives
> - Explore how to optimise ML models with
>   a much larger number of parameters
> - Learn strategies for dealing with large models and
>   large parameter spaces

# Gradient boosting methods: XGBoost

In [1]:
# when delivering live coding, these libraries and all code in this cell have already been loaded
import os
current_work_dir = %pwd
MLPy_ROOT = current_work_dir.split('course')[0]
NOTEBOOK_FOLDER = os.path.join(MLPy_ROOT, 'course',  'notebooks')
MODELS_FOLDER = os.path.join(NOTEBOOK_FOLDER, 'models')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
import pickle

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score,  mean_absolute_error

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler

from sklearn.utils import resample


# Set up plotting options for seaborn and matplotlib
sns.set_context('notebook') 
sns.set_style('ticks') 
%matplotlib inline
plt.rcParams['figure.figsize'] = (9, 6)

# load from previous lessons
cached_files = [
    'ames_train_y.pickle','ames_test_y.pickle',
    'ames_train_X.pickle','ames_test_X.pickle',
    'predictors.pickle','ames_ols_all.pickle',
    'ames_ridge.pickle','ames_lasso.pickle', 
    'ames_enet.pickle',
    'ames_pcr.pickle', 'ames_plsr.pickle',
    'ames_rf.pickle', 'ames_knn.pickle'
]

for file in cached_files:
    with open(os.path.join(MODELS_FOLDER, file), 'rb') as f:
        objectname = file.replace('.pickle', '')
        exec(objectname + " = pickle.load(f)")
        f.close()

## 
def assess_model_fit(models,
                     model_labels, 
                     datasetX, 
                     datasetY):
    columns = ['RMSE', 'R2', 'MAE']
    rows = model_labels
    results = pd.DataFrame(0.0, columns=columns, index=rows)
    for i, method in enumerate(models):
        tmp_dataset_X = datasetX
        # while we build the model and predict on the log10Transformed 
        # sale price, we display the error in dollars as that makes more sense
        y_pred=10**(method.predict(tmp_dataset_X))
        results.iloc[i,0] = np.sqrt(mean_squared_error(10**(datasetY), y_pred))
        results.iloc[i,1] = r2_score(10**(datasetY), y_pred)
        results.iloc[i,2] = mean_absolute_error(10**(datasetY), y_pred)
    return results.round(3)

In [2]:
import xgboost as xgb
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import root_mean_squared_error

## Gradient boosting and XGBoost

Models like `XGBoost` can achieve high accuracy
on a wide range of datasets, but they are complex
models with a large number of hyperparameters
governing their behaviour.

With a large number of hyperparameters:

- It can be difficult to "manually tune" models by inspecting
  how performance changes across different values of the parameter
- The search space gets very large: if you have $n_1$ values for one parameter, then $n_2$ and $n_3$ values for other parameters, a grid
  search has to test $n_1 \times n_2 \times n_3$ parameter combinations

To tune models with many hyperparameters, you can:

* Use the university HPC (Artemis) to run a grid search. Since this works best when you can split a task into smaller individual jobs, you can construct
  a grid of parameters, and for each individual parameter combination
  (and possibly each cross validation fold!),
  pass them to a script that fits the training data to that combination
  and returns the score
* Use a parameter tuning algorithm that can explore the search space automatically, and suggest/generate parameters that are closer
  to the optimal parameters. Examples include [hyperopt](http://hyperopt.github.io/hyperopt/) and [optuna](https://optuna.org/)

To demonstrate this, we will use the [hyperopt](http://hyperopt.github.io/hyperopt/) package to optimize the parameters.

## Automatic parameter tuning with hyperopt

Tuning a model with `hyperopt` requires you to define the search
space a bit differently to the grid search: instead of providing
specific values, you have to define the random distributions
to sample from during tuning.

A simple way to do this is to just specify `uniform` distributions for any parameters that can take fractional values (floats) and `randint` distributions for parameters that require integers.

The difficult part is determining a range that's reasonable for each parameter - this requires some understanding of the model and how each parameter is used.

> ### ⚠️ Challenge: XGBoost parameters
> Review the documentation about XGBoost's [parameters](https://xgboost.readthedocs.io/en/stable/parameter.html#parameters-for-tree-booster).
> Do the values defined here for the parameter search look sensible?
> If you don't have enough knowledge to judge this yourself, where
> could you look for guidance?

In [3]:
space = {
    'max_depth': hp.randint("max_depth", 1, 5),
    'gamma': hp.uniform('gamma', 0, 1),
    'reg_alpha' : hp.uniform('reg_alpha', 0, 50),
    'reg_lambda' : hp.uniform('reg_lambda', 10,100),
    'colsample_bytree' : hp.uniform('colsample_bytree', 0, 1),
    'min_child_weight' : hp.uniform('min_child_weight', 0, 5),
    'n_estimators': hp.randint("n_estimators", 50, 1000),
    'learning_rate': hp.uniform('learning_rate', 0, .15),
    'max_bin' : hp.randint("max_bin", 50, 500)
}

`hyperopt` also requires you to define a function to fit the model
for each parameter combination, and return the error metric. This is a bit more manual work than previous models, where `sklearn` has
tools to automatically choose parameters based on an error metric.

In [4]:
def hyperparameter_tuning(space):
    model = xgb.XGBRegressor(**space, objective='reg:squarederror')
    # NOTE: using early stopping like this to prevent overfitting
    #   may require a "validation" dataset, separate from
    #   both the training and test set. We just use the test
    #   set here for illustration
    evaluation = [(ames_test_X, ames_test_y)]
    model.fit(ames_train_X, ames_train_y, 
              eval_set=evaluation, 
              eval_metric="rmse",
              early_stopping_rounds=100,
              verbose=False)

    #Obtain prediction and rmse score.
    pred = model.predict(ames_train_X)
    rmse = root_mean_squared_error(ames_train_y, pred)
    
    #Specify what the loss is for each model.
    return {'loss': rmse, 'status': STATUS_OK, 'model': model}

Once you've defined the search space and the tuning function, you can start running the algorithm to search for the optimal parameters.

For this workshop, we will only run a small number of iterations - definitely not enough to properly explore the parameter space.

More iterations require more computing resources and more time. As a guide, 

In [5]:
n_trials = 20
trials = Trials()
best = fmin(fn=hyperparameter_tuning,
            space=space,
            algo=tpe.suggest,
            max_evals=n_trials,
            trials=trials)

print(best)

100%|███████████████████| 2000/2000 [11:07<00:00,  3.00trial/s, best loss: 0.029975555369995882]
{'colsample_bytree': 0.21717333480526305, 'gamma': 0.00013087736688291036, 'learning_rate': 0.11492620818012601, 'max_bin': 481, 'max_depth': 3, 'min_child_weight': 0.0041109489525955846, 'n_estimators': 951, 'reg_alpha': 0.022904856862194, 'reg_lambda': 83.27009836487424}


In [6]:
best_grid = {}
for key, val in best.items():
    best_grid[key] = [val]
best_grid

{'colsample_bytree': [0.21717333480526305],
 'gamma': [0.00013087736688291036],
 'learning_rate': [0.11492620818012601],
 'max_bin': [481],
 'max_depth': [3],
 'min_child_weight': [0.0041109489525955846],
 'n_estimators': [951],
 'reg_alpha': [0.022904856862194],
 'reg_lambda': [83.27009836487424]}